In [3]:
%load_ext autoreload
%autoreload 2

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import torch
from funcs import run_all_ml_experiments, run_ml_experiment, run_dl_experiment
from msr.training.data.transforms import Flatten, Permute

from msr.models.modules import ClassifierModule
from msr.models.architectures.networks.mlp import MLPClassifier
from msr.models.architectures.networks.cnn import CNNClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ml_transform = Flatten(start_dim=0, end_dim=-1)

In [5]:
def lgbm_model_provider():
    return LGBMClassifier(n_jobs=-1)

def rf_model_provider():
    return RandomForestClassifier(n_jobs=-1)

def lr_model_provider():
    return LogisticRegression(n_jobs=-1)

def mlp_model_provider(num_classes, input_shape):
    input_size = input_shape[0]
    net = MLPClassifier(
        num_classes=num_classes,
        input_size=input_size,
        hidden_dims=[input_size // (2**i) for i in range(1, 4)]
    )
    return ClassifierModule(net=net)

def cnn_model_provider(num_classes, input_shape):
    net = CNNClassifier(
        num_classes=num_classes, 
        dim=1,
        in_channels=12,
        out_channels=[4],
        kernel_size=[11],
        maxpool_kernel_size=[2]
    )
    return ClassifierModule(net=net)

# **Random Forest**

In [53]:
rf_results = run_all_ml_experiments(rf_model_provider)

A Jupyter Widget

whole_signal_waveforms 0.7158476710319519
whole_signal_features 0.8975158929824829
agg_beat_waveforms 0.8873464465141296
agg_beat_features 0.9018481969833374


# **Logistic Regression**

In [54]:
lr_results = run_all_ml_experiments(lr_model_provider)

A Jupyter Widget

whole_signal_waveforms 0.49
whole_signal_features 0.84
agg_beat_waveforms 0.85
agg_beat_features 0.87


/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

# **MLP**

In [8]:
mlp_results = run_dl_experiment("agg_beat_features", mlp_model_provider, ml_transform, ml_transform)
mlp_results['metrics']['val/auroc']

/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | net       | MLPClassifier | 577 K 
1 | criterion | NLLLoss       | 0     
--------------------------------------------
577 K     Trainable params
0         Non-trainable params
577 K     Total params
2.312     Total estimated model params size (MB)


[Parameter containing:
tensor([[-0.0206, -0.0125,  0.0161,  ...,  0.0168,  0.0063,  0.0295],
        [-0.0008,  0.0275, -0.0026,  ..., -0.0172, -0.0181, -0.0264],
        [-0.0216,  0.0032,  0.0164,  ...,  0.0061, -0.0017, -0.0004],
        ...,
        [ 0.0039, -0.0243, -0.0296,  ...,  0.0103,  0.0152, -0.0262],
        [ 0.0156,  0.0094,  0.0197,  ...,  0.0080, -0.0242, -0.0310],
        [-0.0165,  0.0033, -0.0165,  ...,  0.0215,  0.0280, -0.0278]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 1.9780e-02, -3.4932e-03, -1.1202e-02,  1.0865e-02,  5.7630e-03,
         2.8192e-02,  1.8542e-02,  1.6307e-03,  1.8252e-02, -2.1169e-02,
         1.2541e-02,  2.5700e-02,  3.2680e-02, -2.6633e-02, -6.4885e-03,
         1.1153e-02,  1.7467e-02,  3.1970e-02, -2.8249e-02,  2.7227e-02,
        -2.7243e-03, -1.5456e-02, -3.3785e-03,  7.5175e-03, -2.0010e-02,
         2.6150e-02, -1.0684e-03,  1.7054e-03,  2.5239e-02,  2.4588e-02,
        -8.0670e-03, -2.6568e-02,  1.4

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7bf5eb05e0>
Traceback (most recent call last):
  File "/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1430, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.8/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 


A Jupyter Widget

/home/shate/.cache/pypoetry/virtualenvs/msr-xbuxOujG-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

# **CNN**

In [6]:
cnn_transform = Permute(dims=(1, 0))
cnn_results = run_dl_experiment("agg_beat_waveforms", cnn_model_provider, cnn_transform, cnn_transform)
cnn_results['metrics']['val/auroc']

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | net       | CNNClassifier | 565   
1 | criterion | NLLLoss       | 0     
--------------------------------------------
565       Trainable params
0         Non-trainable params
565       Total params
0.002     Total estimated model params size (MB)


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

`Trainer.fit` stopped: `max_epochs=10` reached.


0.6715338826179504

# **LGBM**

In [6]:
lgbm_results = run_all_ml_experiments(lgbm_model_provider)

A Jupyter Widget

whole_signal_waveforms 0.82
whole_signal_features 0.93
agg_beat_waveforms 0.91
agg_beat_features 0.91
